In [1]:
#分组与聚合 (Groupby)
#就是数据分析这道大餐的“主菜”——分类下锅，爆炒出锅。
#这是 Pandas 中最强大的功能，没有之一。掌握了它，你就能真正开始从数据中挖掘洞察了。

#“分割-应用-合并” (Split-Apply-Combine)

In [2]:
#分割 (Split)：根据某个列（如 country），把整个 DataFrame 拆分成一个个小的数据块。比如，所有美国的放一堆，所有日本的放一堆。

#应用 (Apply)：对每一个小数据块，独立地执行一个计算。比如，对美国那堆数一下有多少行，对日本那堆也数一下有多少行。

#合并 (Combine)：把每一个小数据块的计算结果，重新组合成一个新的、更简洁的表格。

#!!!!!groupby() 是所有商业分析报告的核心。

In [3]:
#今日目标：回答我们总监（我）提出的两个核心业务问题：

#1.全球化布局问题：我们的内容主要来自哪些国家？

#2.内容增长趋势问题：我们的内容库是如何随时间增长的？

In [4]:
#1.全球化布局问题：我们的内容主要来自哪些国家？

In [5]:
#1.全球化布局问题：我们的内容主要来自哪些国家？

#第一步：按“国家”分组，统计各国作品数量
import pandas as pd

df = pd.read_csv('netflix_titles.csv')

# 1. 按 'country' 列进行分组
# 2. 选择任意一列（比如 'show_id'）来进行计数 .count()
# 3. .reset_index() 是一个很重要的步骤，它会把分组的列（'country'）变回普通的列
# 4. .sort_values() 按照作品数量进行降序排列
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [6]:
country_count = df.groupby('country')['show_id'].count()
# 1. 按 'country' 列进行分组
# 2. 选择任意一列（比如 'show_id'）来进行计数 .count()
print(len(country_count))
country_count.head()


######   groupby() 的结果，不是一个标准的表格
#####    它的类型是 Series ，是一维数组，country 成了索引 (Index)
#####    Argentina, Australia 这些国家名字，并没有被当作一列数据，而是被“提拔”成了这个 Series 的行标签（索引）。数据的“值”只有一列（show_id），就是作品的数量。


#####   .count() 的唯一工作：点人头，不是做计算

748


country
, France, Algeria                                       1
, South Korea                                           1
Argentina                                              56
Argentina, Brazil, France, Poland, Germany, Denmark     1
Argentina, Chile                                        2
Name: show_id, dtype: int64

In [7]:
country_count = df.groupby('country')['show_id'].count().reset_index()
# 3. .reset_index() 是一个很重要的步骤，它会把分组的列（'country'）变回普通的列
###  .reset_index() 的作用：把“行标签”变回“普通列”，它会做两件非常重要的事情
###  1“解放”索引：它会把当前作为索引的列（在这里是 country）“解放”出来，降级成一个普普通通的数据列
###  2.“重置”索引：它会重新创建一个最标准的、从0开始的数字索引。
###  最终返回的数据类型变成了 DataFrame

country_count.head()

#一句话总结：reset_index() 的作用，就是把 groupby 产生的、把分组键作为索引的特殊 Series，“整形”成一个标准的、所有数据都是普通列的 DataFrame，
#从而方便我们进行下一步的排序、重命名、可视化等所有标准操作

,country,show_id
0,", France, Algeria",1
1,", South Korea",1
2,Argentina,56
3,"Argentina, Brazil, France, Poland, Germany, De...",1
4,"Argentina, Chile",2


In [8]:
country_counts = df.groupby('country')['show_id'].count().reset_index().sort_values(by='show_id', ascending=False)
# 4. .sort_values() 按照作品数量（show_id的数量）这一列进行降序排列
country_counts.head()

,country,show_id
603,United States,2818
251,India,972
506,United Kingdom,419
319,Japan,245
435,South Korea,199


In [9]:
# 为了让列名更有意义，我们重命名一下
country_counts.rename(columns={'show_id': 'content_count'}, inplace=True)

### Why we need inplace=True?为什么我们需要inpalce =True
### 首先remane重命名如果不设置inplce 默认 inplace=False，此时返回了一个新对象，但是没有修改原来的对象
### 所以此时必须用新的变量储存新的改变

### 如果要对原来的对象进行修改，则要用inplace=True 直接替换原对象中的数据

In [10]:
print("作品数量排名前10的国家:")
country_counts.head(10)

作品数量排名前10的国家:


,country,content_count
603,United States,2818
251,India,972
506,United Kingdom,419
319,Japan,245
435,South Korea,199
58,Canada,181
444,Spain,145
162,France,124
342,Mexico,110
150,Egypt,106


In [11]:
# groupby('')[].count().reset_index().sort_values(by='',ascending='')的方式和value_counts的区别

new = df['country'].value_counts()
new.head()

#value_counts()同样也能计算country列的不同country的数量(并且按从大到小排列) 但是返回的是一个series(以country为index的)
#value_counts() 的功能非常专一：它只能做“计数” 这一件事。
#value_counts()一般用来看一眼数量的情况

#而groupby() 是一个更强大、更通用的“瑞士军刀”，而 value_counts() 只是其中一把专门用来“计数”的小刀。
#并且df.groupby(...).count().reset_index()它返回的是一个标准的 DataFrame。
#.count()只数非空的行,统计行数————>只管点名、不管内容 就算['type'].count()，type有很多都相同，结果也和['show_id'].count()的结果一样
#这种格式更“正规”，非常方便我们进行后续的重命名、再次筛选、或者用作可视化（很多可视化库更喜欢接收列名清晰的 DataFrame）。

country
United States     2818
India              972
United Kingdom     419
Japan              245
South Korea        199
Name: count, dtype: int64

In [12]:
#2.内容增长趋势问题：我们的内容库是如何随时间增长的？

In [13]:
#目标：通过 groupby('release_year')，计算出 Netflix 平台上的内容是越来越新，还是越来越旧，以及增长的速度是快是慢。

year_count = df.groupby('release_year')['show_id'].count().reset_index()
# 为什么自动按年份从小到大排序了？？？
# 因为groupby() 的一个“贴心”默认设置：sort=True
# groupby() 函数就像一个非常勤劳的图书管理员，它在默认情况下，会帮你把分组的“类别”（也就是 release_year）自动从小到大排好序。
# 数字从小到大排序，字符从A-Z排序
# 默认为df.groupby('release_year', sort=True)

year_count.rename(columns={'show_id':'content_counts'},inplace=True)

year_count.tail(10)
# .tail(10) 这个函数可以方便地让我们只看一个 DataFrame 的最后10行

,release_year,content_counts
64,2012,237
65,2013,288
66,2014,352
67,2015,560
68,2016,902
69,2017,1032
70,2018,1147
71,2019,1030
72,2020,953
73,2021,592


In [16]:
#小任务

#1.按作品类型 (type) 分组，统计一下 'Movie' 和 'TV Show' 各有多少部？

type_counts = df.groupby('type')['show_id'].count().reset_index()

type_counts.rename(columns={'show_id':'contend_counts'},inplace = True)
#columes=  别忘记加=了
3type_counts.head()

,type,contend_counts
0,Movie,6131
1,TV Show,2676


In [17]:
#2.按评级 (rating) 分组，找出数量最多的前 5 个评级是什么？

rating_counts = df.groupby('rating')['show_id'].count().reset_index().sort_values(by='show_id',ascending=False)

rating_counts.rename(columns={'show_id':'content_counts'},inplace = True)

rating_counts.head()                                                             

,rating,content_counts
11,TV-MA,3207
9,TV-14,2160
12,TV-PG,863
8,R,799
7,PG-13,490
